## support vector machine con grid search

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


#setting up labels for dataset
labels = ['class', 'spec_num', 'eccentr', 'asp_ratio', 'elong', 'solidity', 'stoch_conv', 'iso_factor', 'max_ind_depth', 'lobedness', 'av_intensity', 'av_contr', 'smooth', 'third_mom', 'unif', 'entropy']
#importing data
df = pd.read_csv(r'./leaf/leaf.csv', header = None, names = labels)
# shuffling the dataframe
df = df.sample(frac=1).reset_index()
df = df.iloc[:, 1:17]   # needed to eliminate the old indexes column
#display(df)
#separating y from x
X = df.iloc[:, 2:16]
y = df.iloc[:, 0]

#display(X)

#### prima grid search (più grossolana) con k-fold CV e LOOCV

In [ ]:
k = 5

# 'preparazione' indici di effectivness
# si usa la funzione make_scorer per costruire le metriche che ci servono
b_accuracy = metrics.make_scorer(metrics.balanced_accuracy_score)
recall = metrics.make_scorer(metrics.recall_score, average='weighted')
scorers = {'balanced_accuracy': b_accuracy, 'recall': recall}

# building the range of the regularization parameter (C)
exp = np.arange(-10, 12)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'degree': np.arange(2, 5),
              'decision_function_shape': ['ovo', 'ovr']}

clf_cv = GridSearchCV(svm.SVC(), grid_param, cv=k, scoring=scorers, refit=False)
clf_cv.fit(X, y)

In [ ]:
# showing the mean values of effectivness indexes

results_cv = pd.DataFrame(clf_cv.cv_results_)

display(results_cv.loc[:, ('params', 'mean_test_balanced_accuracy', 'rank_test_balanced_accuracy', 'mean_test_recall', 'rank_test_recall')])

qua sotto visualizziamo prima soltanto le righe con i valori più alti della balanced accuracy (che corrispondono praticamente sempre a quelle con la recall più alta) e poi soltanto quelle con la roc auc ovo più alta (che corrispondono praticamente sempre a quelle con la roc auc ovr più alta).


si hanno sei set di parametri che condividono la prima posizione perché:
1) il kernel migliore è quello lineare, pertanto param_degree è ininfluente rispetto alla costruzione del modello
2) per qualche motivo (da chiarire) decision_function_shape=ovo e decision_function_shape=ovr sono equivalenti

In [ ]:
display(results_cv[results_cv["rank_test_balanced_accuracy"]==1])
display(results_cv[results_cv["rank_test_balanced_accuracy"]==1]["mean_test_balanced_accuracy"])

In [ ]:
# LOOCV

# building the range of the regularization parameter (C)
exp = np.arange(-8, 8)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'degree': np.arange(2, 5),
              'decision_function_shape': ['ovo', 'ovr']}

clf_loocv = GridSearchCV(svm.SVC(), grid_param, cv=LeaveOneOut(), scoring='accuracy')
clf_loocv.fit(X, y)
print(clf_loocv.best_params_)
print(clf_loocv.best_score_)

di seguito un modo alternativo di fare la grid search con cross validation nel quale si può anche calcolare ROC AUC

l'unica differenza con sopra è che chiediamo a svc di calcolare anche la probabilità (necessaria al calcolo di roc_auc)

è mooolto più lento: credo che il procedimento per il calcolo della probabilità sia abbstanza complicato, non come in tree (dove infatti è presente di default)

In [ ]:
k = 5

# 'preparazione' indici di effectivness
# si usa la funzione make_scorer per costruire le metriche che ci servono
auc_ovo = metrics.make_scorer(metrics.roc_auc_score, multi_class='ovo', needs_proba=True, average='weighted')
auc_ovr = metrics.make_scorer(metrics.roc_auc_score, multi_class='ovr', needs_proba=True, average='weighted')
scorers = {'roc_auc_ovo': auc_ovo, 'roc_auc_ovr': auc_ovr}

# building the range of the regularization parameter (C)
exp = np.arange(-8, 12)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf'], 
              'decision_function_shape': ['ovo', 'ovr']}

clf_cv_2 = GridSearchCV(svm.SVC(probability=True), grid_param, cv=k, scoring=scorers, refit=False)
clf_cv_2.fit(X, y)

In [ ]:
# showing the mean values of effectivness indexes

results_cv_2 = pd.DataFrame(clf_cv_2.cv_results_)

display(results_cv_2[results_cv_2["rank_test_roc_auc_ovo"]==1])

#### seconda grid search (più specifica)

visti i buoni risultati ottenuti, facciamo una ricerca più approfondita attorno ai valori di C migliori

In [ ]:
k = 5

# 'preparazione' indici di effectivness
# si usa la funzione make_scorer per costruire le metriche che ci servono
b_accuracy = metrics.make_scorer(metrics.balanced_accuracy_score)
auc_ovo = metrics.make_scorer(metrics.roc_auc_score, multi_class='ovo', needs_proba=True, average='weighted')
scorers = {'balanced_accuracy': b_accuracy, 'roc_auc_ovo': auc_ovo}

# building the range of the regularization parameter (C)
exp = np.arange(2, 7, 0.2)
reg_param = 10.**exp

grid_param = {'C': reg_param}

clf_cv_spec = GridSearchCV(svm.SVC(probability=True, kernel='linear', decision_function_shape='ovo'), grid_param, cv=k, scoring=scorers, refit=False)
clf_cv_spec.fit(X, y)

In [ ]:
results_cv_spec = pd.DataFrame(clf_cv_spec.cv_results_)

display(results_cv_spec[results_cv_spec["rank_test_balanced_accuracy"]==1])
display(results_cv_spec[results_cv_spec["rank_test_balanced_accuracy"]==1]["mean_test_balanced_accuracy"])

l'accuracy migliora di poco rispetto alla grid search più generica